# Объединение таблиц

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys, os
import pathlib
from pathlib import Path
from olist_churn_prediction.paths import SRC_DIR, PROCESSED_DIR, INTERIM_NOTEBOOK_DIR, RAW_DIR
from olist_churn_prediction import feature_processing, feature_engineering

In [3]:
pd.set_option('display.max_columns', None)

## geolocation_data

In [4]:
geolocation_data = pd.read_parquet(INTERIM_NOTEBOOK_DIR / "geolocation_olist_public_dataset_interim.parquet")

In [5]:
display(geolocation_data.head())
print(geolocation_data.shape)
geolocation_data.dtypes

,zip_code_prefix,city,state,lat,lng
0,010,sao_paulo,sp,-23.547808,-46.636124
1,010,sao_paulo,sp,-23.546082,-46.644821
2,010,sao_paulo,sp,-23.540983,-46.642567
3,010,sao_paulo,sp,-23.546759,-46.645771
4,010,sao_paulo,sp,-23.546362,-46.643074


(323016, 5)


zip_code_prefix    string[python]
city               string[python]
state              string[python]
lat                       float32
lng                       float32
dtype: object

## payments_data

In [5]:
payments_data =  pd.read_parquet(INTERIM_NOTEBOOK_DIR / "payments_olist_public_dataset_interim.parquet")

In [6]:
display(payments_data.head())
payments_data.shape

,order_id,installments,sequential,payment_type,value
0,25e8ea4e93396b6fa0d3dd708e76c1bd,1,1,credit_card,65.709999
1,ba78997921bbcdc1373bb41e913ab953,8,1,credit_card,107.779999
2,42fdf880ba16b47b59251dd489d4441a,2,1,credit_card,128.449997
3,771ee386b001f06208a7419e4fc1bbd7,1,1,credit_card,81.160004
4,62fe0bd75deb58982ebf8e53a128a301,1,1,credit_card,15.820000


(97428, 5)

In [7]:
payments_data.isna().sum()

order_id        0
installments    0
sequential      0
payment_type    0
value           0
dtype: int64

In [8]:
# Группировка: множество payment_type по каждому user_id
type_counts = payments_data.groupby("order_id")["payment_type"].nunique()

# Оставляем только те user_id, где один уникальный тип оплаты
consistent_users = type_counts[type_counts == 1].index

# Фильтрация исходного DataFrame
filtered_df = payments_data[payments_data["order_id"].isin(consistent_users)]

print(filtered_df.shape)

(92119, 5)


Количество заказов с одним способом оплаты - 92119, изначальное количество - 97428. Решено убрать признак payment_type для сохранения числа заказов.

In [9]:
payments_data = payments_data.drop('payment_type', axis=1)

Сгруппируем по order_id:

In [10]:
payments_data_grouped = filtered_df.groupby("order_id").agg({
    "value": "sum",
    "installments": "max",
    "sequential": "count"}).reset_index()

In [11]:
display(payments_data_grouped.head())

,order_id,value,installments,sequential
0,0000f01da58c553799200eb71eb0563b,72.900002,1,1
1,00010242fe8c5a6d1ba2dd792cb16214,72.190002,2,1
2,00026c8a4fe2c14efa2afe807921981c,477.279999,1,1
3,000312d02b605a8a630c3a9882793ffb,673.200012,10,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.039993,3,1


In [14]:
#payments_data_grouped.to_parquet(INTERIM_NOTEBOOK_DIR / "payments_grouped_interim.parquet", index=False)

## measures_data

In [12]:
measures_data = pd.read_parquet(INTERIM_NOTEBOOK_DIR / "product_measures_olist_public_dataset_interim.parquet")

In [13]:
display(measures_data.head())
measures_data.shape

,product_id,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00faa46f36261af8bbf3a4d37fa4841b,100.0,21.0,9.0,14.0
1,b497d0032ea073dd58bbd2d7809d25b3,750.0,23.0,5.0,15.0
2,96bd76ec8810374ed1b65e291975717f,200.0,16.0,6.0,19.5
3,506110a1603e772a8ff52ea1f0b0431d,200.0,16.0,13.0,13.0
4,feb593f17c2f51af2932dbba3e99f1cb,50.0,35.0,4.0,26.0


(24439, 5)

In [14]:
measures_data.isna().sum()

product_id           0
product_weight_g     0
product_length_cm    0
product_height_cm    0
product_width_cm     0
dtype: int64

## customers_data

In [15]:
customers_data = pd.read_parquet(INTERIM_NOTEBOOK_DIR / "public_customers_data_interim.parquet")

In [16]:
display(customers_data.head())
customers_data.shape

,customer_id,customer_unique_id
0,0000ca4ff2795842fd6b122c6d974468,ac0e5d7f07043dd32831bdad3b99ad26
1,00010f206878ba01f199aec4237de72e,4ed24aaed4079fe0661c0e4b3b420dff
2,00012a2ce6f8dcda20d059ce98491703,7e0e291c0f4fc2d69208954d26ed0586
3,000161a058600d5901f007fab4c27140,ff1dcb27ea444eddb94ea5fea77b875c
4,00017f1d6cce0d56046219a2cfabcbbb,237130c4b6717031ed62d9d0b16fd94f


(96264, 2)

In [17]:
customers_data.isna().sum()

customer_id           0
customer_unique_id    0
dtype: int64

## sellers_data

In [18]:
sellers_data = pd.read_parquet(INTERIM_NOTEBOOK_DIR / "sellers_olist_public_dataset_interim.parquet")

In [19]:
display(sellers_data.head())
sellers_data.shape

,order_id,product_id,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,0faabac07131235fc5d9d711471cb4db,40b09f33e646d488df2ca6fec4082d50,3d871de0142ce09b7081e2b9d1733cb1,132,campo_limpo_paulista,sp
1,970f41d57d6e21afa7b8c701b09acb95,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,371,ilicinea,mg
2,b674ed44cc3f6a869249421debebe104,bd7cd34fc6d02e730221b11edc354aae,46dc3b2cc0980fb8ec44634e21d2718e,222,rio_de_janeiro,rj
3,cbe9eae36605cf2bd005c6bc1ae5f864,0e95d6eef2bedaf4ecf3c33f78199059,dc4a0fc896dc34b0d5bfec8438291c80,149,ibitinga,sp
4,f562f8a4adf5a459176f7170d0da220d,9a3a44b7cc16f2592b2110e4205edf97,5a8e7d5003a1f221f9e1d6e411de7c23,130,campinas,sp


(98981, 6)

In [20]:
sellers_data.isna().sum()

order_id                  0
product_id                0
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

## public_data

In [4]:
public_data = pd.read_parquet(INTERIM_NOTEBOOK_DIR / "public_data_interim.parquet")

In [5]:
display(public_data.head())
print(public_data.shape)

,order_id,order_products_value,order_freight_value,order_items_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_id,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_creation_date,review_answer_timestamp,product_category_name_english
0,b95df3cef5297e79ef709ba256518f6f,349.899994,13.840000,1,2017-01-31 17:19:01,2017-02-01 02:41:21.549551,2017-03-15,2017-02-06 11:04:24.154259,297dcf281305aabfee256d313076af2b,guaratuba,pr,832,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,2017-02-07,2017-02-09 02:37:37,health_beauty
1,59af46052a799e80e2f0c665c587731d,15.000000,15.100000,1,2017-09-09 19:52:54,2017-09-10 20:03:31.535281,2017-10-02,2017-09-13 20:17:41.296915,b7563d94162ed0aa3f8d9cada4bcc536,uberlandia,mg,384,44,1428,2,ae5cad88462eb7b7b61401e31c45618e,59af46052a799e80e2f0c665c587731d,5,2017-09-14,2017-09-15 03:43:47,computers_accessories
2,a3e6136894621db402a772c6bc72a12a,238.899994,18.000000,1,2017-01-30 17:00:09,2017-01-30 17:31:25.438253,2017-03-07,2017-02-06 15:43:04.758566,f102cb15ec41ca30fa2902da06495f6b,itapevi,sp,066,55,637,1,0c9ff9d8ed9b9bdd825487b3a66e05f5,a3e6136894621db402a772c6bc72a12a,5,2017-02-07,2017-02-10 14:18:53,auto
3,b675ea5a618922f6e679e30531b8957b,29.990000,18.230000,1,2018-03-11 18:18:36,2018-03-11 18:30:37.931962,2018-04-03,2018-04-03 20:36:43.778451,e33c250311160298e5b457ca52720a35,goiania,go,743,55,617,1,ad0a798e7941f3a5a2fb8139cb62ad78,b675ea5a618922f6e679e30531b8957b,4,2018-04-04,2018-04-05 02:52:31,bed_bath_table
4,195a8be6794c487fe6cfbb97b7c61902,295.989990,47.650002,1,2017-04-20 08:01:08,2017-04-25 08:05:40.405383,2017-05-24,2017-05-04 18:47:45.721758,8a8f7c9a4450f17ba3ee25a5ba821bc7,recife,pe,511,49,558,2,eaf2046d4c87809247a30050ea13df03,195a8be6794c487fe6cfbb97b7c61902,5,2017-05-05,2017-05-08 15:20:18,furniture_decor


(97578, 21)


In [6]:
public_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97578 entries, 0 to 97577
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       97578 non-null  string        
 1   order_products_value           97578 non-null  float32       
 2   order_freight_value            97578 non-null  float32       
 3   order_items_qty                97578 non-null  Int64         
 4   order_purchase_timestamp       97578 non-null  datetime64[ns]
 5   order_aproved_at               97578 non-null  datetime64[ns]
 6   order_estimated_delivery_date  97578 non-null  datetime64[ns]
 7   order_delivered_customer_date  97578 non-null  datetime64[ns]
 8   customer_id                    97578 non-null  string        
 9   customer_city                  97578 non-null  string        
 10  customer_state                 97578 non-null  string        
 11  customer_zip_co

In [25]:
display(public_data[public_data['order_id'] == '5a3b1c29a49756e75f1ef513383c0c12'])

,order_id,order_products_value,order_freight_value,order_items_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_id,customer_city,customer_state,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_creation_date,review_answer_timestamp,product_category_name_english
3640,5a3b1c29a49756e75f1ef513383c0c12,72.900002,2.660000,1,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,31,151,1,6a0713a63e990052795623e76cc2995e,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-31,2017-10-31 21:03:31,bed_bath_table
37312,5a3b1c29a49756e75f1ef513383c0c12,99.980003,14.200000,2,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,40,162,1,99444c1e11fc38755dae86f5c67d3d1a,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-23,2017-10-24 10:44:02,bed_bath_table
45061,5a3b1c29a49756e75f1ef513383c0c12,99.980003,14.200000,2,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,40,162,1,99444c1e11fc38755dae86f5c67d3d1a,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-31,2017-10-31 21:03:31,bed_bath_table
52338,5a3b1c29a49756e75f1ef513383c0c12,58.900002,2.660000,1,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,26,212,2,17d8b8e9c21ec0fc5b33724e32fe422e,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-31,2017-10-31 21:03:31,bed_bath_table
53362,5a3b1c29a49756e75f1ef513383c0c12,139.800003,67.440002,2,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,34,308,1,d25bbb182b51d1594fe3b54a27ae143e,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-23,2017-10-24 10:44:02,bed_bath_table
58651,5a3b1c29a49756e75f1ef513383c0c12,58.900002,2.660000,1,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,26,212,2,17d8b8e9c21ec0fc5b33724e32fe422e,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-23,2017-10-24 10:44:02,bed_bath_table
77024,5a3b1c29a49756e75f1ef513383c0c12,185.699997,35.939999,3,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,34,216,2,399f54c06661d06ca45c6e24ed238e0e,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-23,2017-10-24 10:44:02,bed_bath_table
79507,5a3b1c29a49756e75f1ef513383c0c12,185.699997,35.939999,3,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,34,216,2,399f54c06661d06ca45c6e24ed238e0e,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-31,2017-10-31 21:03:31,bed_bath_table
79549,5a3b1c29a49756e75f1ef513383c0c12,72.900002,2.660000,1,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,31,151,1,6a0713a63e990052795623e76cc2995e,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-23,2017-10-24 10:44:02,bed_bath_table
91684,5a3b1c29a49756e75f1ef513383c0c12,157.800003,15.970000,2,2017-10-17 13:06:29,2017-10-18 13:06:21.646633,2017-11-03,2017-10-22 14:43:54.999785,be1c4e52bb71e0c54b11a26b8e8d59f2,sao_paulo,sp,080,29,263,1,e95fb3a80ca294f39de925b6428f3a32,5a3b1c29a49756e75f1ef513383c0c12,1,2017-10-31,2017-10-31 21:03:31,bed_bath_table


In [8]:
display(public_data[['order_id', 'customer_id']].sort_values(by='order_id'))

,order_id,customer_id
17358,0000f01da58c553799200eb71eb0563b,aaf96a43700cc28f3d1f6115e9bf311e
69538,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a
44142,00026c8a4fe2c14efa2afe807921981c,a7785a0278913274fddadfb4010eb59e
70168,000312d02b605a8a630c3a9882793ffb,d54487c6cb43a7ae702eac4adc1d9224
86530,000330af600103828257923c9aa98ae2,fad0bbc7d520cbb3c5b67d4d4b546826
...,...,...
28192,fffb9224b6fc7c43ebb0904318b10b5f,4d3abb73ceb86353aeadbe698aa9d5cb
25808,fffc6722901386c66d5f8009639fcb24,2e899e8140c36d1792eeaac96d9d058b
50093,fffe18544ffabc95dfada21779c9644f,b5e6afd5a41800fdf401e0272ca74655
88183,fffe7967a0e803b9331617191f6c7ed7,0786576d0923b748d5ac1e4f03e85a1d


Нужно сгруппировать public_data по order_id последующего использования:

In [26]:
# Найти числовые признаки, которые агрегируем как сумму
sum_cols = ['order_products_value', 'order_freight_value', 'order_items_qty', 'product_photos_qty']
mean_cols = ['product_name_lenght', 'product_description_lenght']
min_cols = ['review_creation_date', 'review_answer_timestamp']
# Остальные признаки, которые одинаковые в группе
agg_cols = set(sum_cols + mean_cols + min_cols + ['order_id'])
other_cols = [col for col in public_data.columns if col not in agg_cols]

# Создаем словарь агрегации
agg_dict = {col: "sum" for col in sum_cols}
agg_dict.update({col: "mean" for col in mean_cols})
agg_dict.update({col: "min" for col in min_cols})
agg_dict.update({col: "first" for col in other_cols})

# Применяем группировку
public_data_grouped = public_data.groupby("order_id").agg(agg_dict).reset_index()

In [90]:
#public_data_grouped.to_parquet(INTERIM_NOTEBOOK_DIR / "public_grouped_interim.parquet")

Для анализа расстояний между покупателем и продавцом нам не хватает полного zip_code. В нашем случае имеется лишь префикс в виде трех цифр, что не подходит. Нужны все 5 цифр. geolocation_data не может быть использована в данной ситуации.

In [91]:
public_data_grouped = pd.read_parquet(INTERIM_NOTEBOOK_DIR / "public_grouped_interim.parquet")

In [65]:
len(public_data_grouped['order_id'].unique())

93900

In [93]:
payments_grouped = pd.read_parquet(INTERIM_NOTEBOOK_DIR / "payments_grouped_interim.parquet")

In [94]:
len(payments_grouped['order_id'].unique())

90825

In [95]:
public_and_payments = public_data_grouped.merge(payments_grouped, how='left', on='order_id')

In [69]:
len(public_and_payments['order_id'].unique())

93900

In [97]:
public_and_payments.isna().sum()

order_id                            0
order_products_value                0
order_freight_value                 0
order_items_qty                     0
product_photos_qty                  0
product_name_lenght                 0
product_description_lenght          0
review_creation_date                0
review_answer_timestamp             0
order_purchase_timestamp            0
order_aproved_at                    0
order_estimated_delivery_date       0
order_delivered_customer_date       0
customer_id                         0
customer_city                       0
customer_state                      0
customer_zip_code_prefix            0
product_id                          0
review_id                           0
review_score                        0
product_category_name_english       0
value                            5337
installments                     5337
sequential                       5337
dtype: int64

Пропуски появились вследствие объединения из-за отсутствия платежей за некоторые товары. Логично заполнить нулями (сумма платежа равна нулю если платежа еще не было).

In [98]:
public_and_payments = public_and_payments.fillna(0)

In [99]:
public_and_payments.shape

(93900, 24)

In [100]:
public_and_payments_and_measures = public_and_payments.merge(measures_data, how='left', on='product_id')

In [102]:
public_and_payments_and_measures.shape

(93900, 28)

In [103]:
public_and_payments_and_measures.isna().sum()

order_id                          0
order_products_value              0
order_freight_value               0
order_items_qty                   0
product_photos_qty                0
product_name_lenght               0
product_description_lenght        0
review_creation_date              0
review_answer_timestamp           0
order_purchase_timestamp          0
order_aproved_at                  0
order_estimated_delivery_date     0
order_delivered_customer_date     0
customer_id                       0
customer_city                     0
customer_state                    0
customer_zip_code_prefix          0
product_id                        0
review_id                         0
review_score                      0
product_category_name_english     0
value                             0
installments                      0
sequential                        0
product_weight_g                 18
product_length_cm                18
product_height_cm                18
product_width_cm            

Для каких-то товаров в public_data нет информации из measures_data о размерах. Удалим такие строки.

In [104]:
public_and_payments_and_measures = public_and_payments_and_measures.dropna()

In [105]:
public_and_payments_and_measures_and_customers = public_and_payments_and_measures.merge(customers_data, how='left', on='customer_id')

In [107]:
public_and_payments_and_measures_and_customers.isna().sum()

order_id                         0
order_products_value             0
order_freight_value              0
order_items_qty                  0
product_photos_qty               0
product_name_lenght              0
product_description_lenght       0
review_creation_date             0
review_answer_timestamp          0
order_purchase_timestamp         0
order_aproved_at                 0
order_estimated_delivery_date    0
order_delivered_customer_date    0
customer_id                      0
customer_city                    0
customer_state                   0
customer_zip_code_prefix         0
product_id                       0
review_id                        0
review_score                     0
product_category_name_english    0
value                            0
installments                     0
sequential                       0
product_weight_g                 0
product_length_cm                0
product_height_cm                0
product_width_cm                 0
customer_unique_id  

In [108]:
public_and_payments_and_measures_and_customers_and_sellers = public_and_payments_and_measures_and_customers.merge(sellers_data, how='left', on=['order_id', 'product_id'])

In [110]:
public_and_payments_and_measures_and_customers_and_sellers.isna().sum()

order_id                         0
order_products_value             0
order_freight_value              0
order_items_qty                  0
product_photos_qty               0
product_name_lenght              0
product_description_lenght       0
review_creation_date             0
review_answer_timestamp          0
order_purchase_timestamp         0
order_aproved_at                 0
order_estimated_delivery_date    0
order_delivered_customer_date    0
customer_id                      0
customer_city                    0
customer_state                   0
customer_zip_code_prefix         0
product_id                       0
review_id                        0
review_score                     0
product_category_name_english    0
value                            0
installments                     0
sequential                       0
product_weight_g                 0
product_length_cm                0
product_height_cm                0
product_width_cm                 0
customer_unique_id  

In [111]:
public_and_payments_and_measures_and_customers_and_sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93882 entries, 0 to 93881
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       93882 non-null  string        
 1   order_products_value           93882 non-null  float32       
 2   order_freight_value            93882 non-null  float32       
 3   order_items_qty                93882 non-null  Int64         
 4   product_photos_qty             93882 non-null  Int64         
 5   product_name_lenght            93882 non-null  Float64       
 6   product_description_lenght     93882 non-null  Float64       
 7   review_creation_date           93882 non-null  datetime64[ns]
 8   review_answer_timestamp        93882 non-null  datetime64[ns]
 9   order_purchase_timestamp       93882 non-null  datetime64[ns]
 10  order_aproved_at               93882 non-null  datetime64[ns]
 11  order_estimated

In [112]:
public_and_payments_and_measures_and_customers_and_sellers.to_parquet(INTERIM_NOTEBOOK_DIR / "merged_df.parquet")